
# TensorFlow Object Detection: Подготовка данных для обучения

https://github.com/tensorflow/models/tree/master/research/object_detection

### Монтирование Google Drive
Для данного демо нам понадобятся несколько ноутбуков, которые должны работать с одними и теме же данными. Поэтому, чтобы все ноутбуки имели доступ к нужным данным, нам будет необходимо подключить диск Google Drive и сохранять все данные на нём (включая данные, скачанные из интернета).

Для монтирования диска нужно выполнить данный блок, перейти по ссылке, получить код, скопировать его в поле ниже (в этом блоке) и нажать Enter

После монтирования диск будет находиться здесь: `/content/drive/My Drive`

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
pip install tf_slim

     |████████████████████████████████| 352 kB 7.1 MB/s 


In [3]:
pip install lvis

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Рабочая директория
Все данные будем хранить в директории `/content/drive/My Drive/tf_od_demo` (TensorFlow Object Detection Demo)

При первом запуске создадим директорию (если её еще не существует), в противном случае надо заменить True на False.

При последующих подключениях к диску (в том числе в других ноутбуках) директорию создавать не надо, в ней уже будут сохранены все данные, которые мы туда поместили.

In [5]:
if True:
    !mkdir "/content/drive/My Drive/tf_od_demo"
%cd "/content/drive/My Drive/tf_od_demo"

/content/drive/My Drive/tf_od_demo


### Подготовка библиотеки `object_detection`
Библиотека `object_detection` находится в репозитории `tensorflow/models` в разделе `research`

Необходимо склонировать код библиотеки и сконфигурировать модели (сбилдить прото модели).

Этот шаг нужно сделать один раз (не повторять, если папка `models` уже находится в текущей директории).

Подробнее: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md

In [6]:
if True:
  
    !git clone https://github.com/tensorflow/models
    !cd models/research && protoc object_detection/protos/*.proto --python_out=.
    !cd models/research && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim && python object_detection/builders/model_builder_test.py

Cloning into 'models'...
remote: Enumerating objects: 59177, done.
remote: Counting objects: 100% (648/648), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 59177 (delta 412), reused 558 (delta 335), pack-reused 58529
Receiving objects: 100% (59177/59177), 573.64 MiB | 17.73 MiB/s, done.
Resolving deltas: 100% (41044/41044), done.
Checking out files: 100% (2579/2579), done.


### Загрузка библиотек
Загрузка TensorFlow и других библиотек. Кроме того, загрузка модуля `dataset_util` из пакета `object_detection`, который будет нужен для создания датасета в нужном формате.

In [7]:
import pandas as pd
import os
from PIL import Image

import tensorflow as tf

import sys
sys.path.insert(0, 'models/research')

from object_detection.utils import dataset_util

### Функция для создания одного обучающего образца
В этой функции создаётся экземпляр класса `tf.train.Example`, который соответствует одной обучающей картике. Ей могут соответствовать несколько ground-truth баундинг боксов. Однако, конкретно в данном примере на картинке есть строго один бокс. В противном случае списки `xmins`, `xmaxs`, `ymins`, `ymaxs`, `classes_text`, `classes` должны иметь соответствующее количество элементов ( = кол-ву боксов на данной картинке).

Создавать экземпляры класса `tf.train.Example` можно произвольным способом. В данном примере на вход в функцию подаётся строка из CSV файла (`annot.csv`). Главное -- заполнить соовтестсвующие поля словаре `feature={...}`

Подробнее: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md

In [8]:
def create_tf_example(example):
  
    img_fpath = os.path.join('my_data2', example.id)
    img = Image.open(img_fpath)
    height = img.size[1]
    width = img.size[0]
    filename = str.encode(example.id)
    with open(img_fpath, mode='rb') as f:
        encoded_image_data = f.read()
    image_format = b'jpeg'

    # List of normalized left x coordinates in bounding box (1 per box)
    xmins = [example.xmin1 / float(width), example.xmin2 / float(width)] 
    # List of normalized right x coordinates in bounding box # (1 per box)
    xmaxs = [example.xmax1 / float(width), example.xmax2 / float(width)] 
    # List of normalized top y coordinates in bounding box (1 per box)
    ymins = [example.ymin1 / float(height), example.ymin2 / float(height)] 
    # List of normalized bottom y coordinates in bounding box # (1 per box)
    ymaxs = [example.ymax1 / float(height), example.ymax2 / float(height)] 
    # List of string class name of bounding box (1 per box)
    classes_text = [b'Dog', b'Stump']
    # List of integer class id of bounding box (1 per box)
    classes = [2]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

### Чтение CSV файла с разметкой
В данном файле представлена разметка обучающих изображений. Сам файл и его формат показаны лишь для примера, они никак не связаны с библиотекой `object_detection`. Наша финальная цель -- создать датасет в формате `TFRecord`, состоящий из экземпляров `tf.train.Example`.

---

В данном примере формат файла annot.csv следующий (один бокс на файл):

id,xmin,ymin,xmax,ymax

1.jpg,261,260,601,615

2.jpg,130,429,401,734

...

---

Перед запуском этого блока загрузите необходимые данные (папка `my_data`) в текущую рабочую директорию (tf_od_demo). Один из вариантов, как это можно сделать, это загрузить архив `my_data.7z`, а затем разархивировать его с помощью команды:

`!7z x my_data.7z`

In [44]:
if True:
  !7z x my_data2.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 434406 bytes (425 KiB)

Extracting archive: my_data2.7z
--
Path = my_data2.7z
Type = 7z
Physical Size = 434406
Headers Size = 521
Method = LZMA2:19
Solid = +
Blocks = 1

  0%      0% 9 - my_data2/2.jpg                       Everything is Ok

Folders: 1
Files: 20
Size:       452216
Compressed: 434406


In [45]:
annot = pd.read_csv('my_data2/annot2.csv')
annot.head()

,id,xmin1,ymin1,xmax1,ymax1,xmin2,ymin2,xmax2,ymax2
0,1.jpg,77,272,474,654,463,349,690,602
1,2.jpg,99,308,485,790,483,387,717,686
2,3.jpg,14,305,491,733,483,392,711,673
3,4.jpg,25,163,455,681,474,248,711,583
4,5.jpg,14,351,453,768,474,409,706,705


### Создание TFRecord
Здесь мы создаём финальный датасет в формате `TFRecord`, который необходим для запуска обучения TF Object Detection. 

В цикле по всем обучающим образцам создаем `TF Example` и записываем его в `TF Record`.

In [46]:
writer = tf.python_io.TFRecordWriter('my_data2/train_data.record')

for idx, row in annot.iterrows():
    tf_example = create_tf_example(row)
    writer.write(tf_example.SerializeToString())

writer.close()